### All necessary imports

In [ ]:
%matplotlib inline
import utils2
from keras.models import Model
import keras.backend as K
import numpy as np

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave
from scipy.ndimage import gaussian_filter
from keras import metrics
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt

from vgg16_avg import VGG16_Avg

In [ ]:
utils2.limit_mem() #comment out if not using shared GPU resources

## Load content image


In [ ]:
path = "/home/ubuntu/artisticTransf/images/*"
imgs = glob(path)
len(imagenet_imgs)

In [ ]:
img = Image.open(imagenet_imgs[0])
img

In [ ]:
size = img.size
size

## Load style image

In [ ]:
path = 

In [ ]:
sty = Image.open('/home/ubuntu/artisticTransf/images/insertImage').resize(size, Image.ANTIALIAS) #need to match size with content image

## Preprocessing for VGG and Imagenet files

In [ ]:
rgb_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32)
preproc = lambda x: np.array(x - rgb_mean)[:, :, :, ::-1]

In [ ]:
deproc = lambda x, shape: np.clip(x.reshape(shape)[:, :, :, ::-1] + rgb_mean, 0, 255)

In [ ]:
#expanding dimensions to 4-dimensional tensors for keras
img_arr = preproc(np.expand_dims(img, 0))
sty_arr = preproc(np.expand_dims(sty, 0))
shape = img_arr.shape; shape

## Filling variables in models

In [ ]:
model = VGG16_Avg(include_top=False, input_shape=shape[1:])
name2output = {l.name: l.output for l in model.layers}
outputs = [name2output['block{}_conv2'.format(o)] for o in range(1,6)]


In [ ]:
model = Model(model.input, outputs)
cont_targs = [K.variable(o) for o in model.predict(img_arr)]
sty_targs = [K.variable(o) for o in model.predict(sty_arr)]

## Defining Loss and Gradient Functions


In [ ]:
def gram_mat(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    return K.dot(features, K.transpose(features)) / x.get_shape().num_elements()

In [ ]:
def sty_loss(targ, layer): return K.mean(metrics.mse(gram_mat(targ), gram_mat(layer)))

In [ ]:
cont_loss = sum(K.mean(metrics.mse(cont_targ[0], cont_outp[0])) 
                for cont_targ, cont_outp in zip(cont_targs, outputs))
style_loss = sum(sty_loss(sty_targ[0], sty_outp[0]) 
                 for sty_targ, sty_outp in zip(sty_targs, outputs))

In [ ]:
loss = 0.6*cont_loss + style_loss #reducing weight of content
grad = K.gradients(loss, model.input)
fn = K.function([model.input], [loss]+grad)

In [ ]:
class Evaluator(object): #need to separate the loss and gradients
    def __init__(self, func, shape): self.func, self.shp = func, shape
    
    def loss(self, x):
        loss_, self.grads_ = self.func([x.reshape(self.shp)])
        return loss_.astype(np.float64)
    
    def grads(self, x): return self.grads_.flatten().astype(np.float64)

In [ ]:
eval_obj = Evaluator(fn, shape)

## Creating initializing noise


In [ ]:
noise = lambda shape: np.random.uniform(0, 225, size=shape)
x = noise(shape)
plt.imshow(x[0].astype('uint8'));

In [ ]:
def train(x, shape, eval_obj, iters):
    for i in range(iters):
        x, min_val, info = fmin_l_bfgs_b(eval_obj.loss, x.flatten(), 
                                         fprime=eval_obj.grads, maxfun=20)
        np.clip(x, -127, 127)
        print('Current loss: {}'.format(min_val))
        imsave(f'./results/res_at_iteration_{i}.png', deproc(x.copy(), shape)[0])
    return x

In [ ]:
iters = 20 #define your own iterators
x = train(x, shape, eval_obj, iters)


In [ ]:
result = Image.open('./results/res_at_iteration_19.png')
result

## Can add further display files here
